Utwórz lub pobierz plik z nieregularnymi zakłóceniami (np. odgłosy psów), zmniejsz
jego amplitudę, tak by maksymalna amplituda zakłóceń była połową maksymalnej amplitudy oryginalnego sygnału, a następnie dodaj zakłócenia do próbek 100
próbek. Przetestuj system uwierzytelniania na tak zakłóconych próbkach.

# Imports

In [1]:
import json
import os

import numpy as np

from experiments.prepare_experiments import prepare_experiment_5
from scripts.voice_recognition_system import VoiceRecognitionSystem

C:\Virtual_Environments\venv_BIO_3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define File Paths

In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_directory = os.path.join(parent_directory, "data")

database_path = os.path.join(data_directory, "database")

incoming_authorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "authorized_users"
)
incoming_unauthorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "unauthorized_users"
)

In [3]:
experiment_name = "experiment_5"
experiment_path = os.path.join(data_directory, experiment_name)

experiment_authorized_path = os.path.join(experiment_path, "authorized")
experiment_unauthorized_path = os.path.join(experiment_path, "unauthorized")

In [4]:
results_path = os.path.join(parent_directory, "results", experiment_name)

if not os.path.exists(results_path):
    os.makedirs(results_path, exist_ok=True)

In [5]:
irregular_noise_path = os.path.join(data_directory, "electric-saw-aka-pandemia.wav")

# Initialize System

In [6]:
voice_recognition_system = VoiceRecognitionSystem(
    database_path, acceptance_threshold=0.7, allow_brute_force=False
)

Initializing database: 100%|██████████| 100/100 [00:20<00:00,  4.93it/s]


# Prepare Experiment

In [7]:
prepare_experiment_5(
    experiment_path,
    incoming_authorized_users_path,
    incoming_unauthorized_users_path,
    100,
    irregular_noise_path,
)

# Conduct Experiment

### Without brute force

In [8]:
df_authorized_users = voice_recognition_system.verify_multiple_users(
    incoming_users_path=experiment_authorized_path,
)

Veryfing multiple users: 100%|██████████| 40/40 [00:13<00:00,  3.06it/s]


In [9]:
df_unauthorized_users = voice_recognition_system.verify_multiple_users(
    incoming_users_path=experiment_unauthorized_path,
)

Veryfing multiple users: 100%|██████████| 41/41 [00:10<00:00,  3.87it/s]


### With brute force

In [10]:
voice_recognition_system.allow_brute_force = True

In [11]:
df_unauthorized_users_brute_force = voice_recognition_system.verify_multiple_users(
    incoming_users_path=experiment_unauthorized_path
)

Veryfing multiple users: 100%|██████████| 41/41 [00:03<00:00, 11.39it/s]


# Calculating metrics

In [12]:
configurations = {
    "default": (df_authorized_users, df_unauthorized_users),
    "brute_force": (
        df_authorized_users,
        df_unauthorized_users_brute_force,
    ),
}

In [13]:
for configuration_name in configurations.keys():
    df_authorized_users, df_unauthorized_users = configurations[configuration_name]

    false_acceptance_rate, false_rejection_rate = (
        voice_recognition_system.calculate_far_frr(
            df_authorized_users, df_unauthorized_users
        )
    )
    true_negative, false_positive, false_negative, true_positive = (
        voice_recognition_system.calculate_ROC_curve(
            df_authorized_users,
            df_unauthorized_users,
            roc_curve_path=os.path.join(
                results_path, f"ROC_curve_{configuration_name}.png"
            ),
        )
    )
    accuracy = (true_positive + true_negative) / (
        true_positive + true_negative + false_positive + false_negative
    )
    access_granted_rate = voice_recognition_system.calculate_access_granted_rate(
        df_authorized_users
    )
    results_dict = {
        "FAR": np.round(false_acceptance_rate, 3),
        "FRR": np.round(false_rejection_rate, 3),
        "TN": true_negative,
        "FP": false_positive,
        "FN": false_negative,
        "TP": true_positive,
        "ACC": np.round(accuracy, 3),
        "AGR": np.round(access_granted_rate, 3),
    }

    # convert int64 values to native Python integers
    for key, value in results_dict.items():
        results_dict[key] = int(value) if isinstance(value, np.int64) else value

    with open(
        os.path.join(results_path, f"results_{configuration_name}.json"), "w"
    ) as json_file:
        json.dump(results_dict, json_file, indent=4)